In [17]:
import psycopg2
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import random
from flask import Flask, jsonify 
from flask_cors import CORS

In [18]:
app = Flask(__name__)
CORS(app)

In [19]:
def connect_db():
    conn = psycopg2.connect(
    # Check these parameters
    host="ep-damp-dust-a11bi2qp-pooler.ap-southeast-1.aws.neon.tech", 
    port="5432",
    database="neondb",
    user="neondb_owner",
    password="Ijov0mkf6Ohl",
    sslmode="require"
    )

    return conn

In [20]:
def create_enhanced_time_series_plot(df, city_name):
    # Ensure 'Date' column is in datetime format
    df['Date'] = pd.to_datetime(df['Date']).dt.day
    
    # Calculate rolling average
    df['Rolling_Avg'] = df['Count'].rolling(window=7).mean()  # 7-day rolling average

    # Create the main line plot
    fig = go.Figure()

    # Add the main line plot
    fig.add_trace(go.Scatter(
        x=df['Date'],
        y=df['Count'],
        mode='lines+markers',
        name='Daily Count',
        line=dict(width=2),
        marker=dict(size=6)
    ))

    # Add the rolling average line (7-day rolling)
    fig.add_trace(go.Scatter(
        x=df['Date'],
        y=df['Rolling_Avg'],
        mode='lines',
        name='7-day Rolling Average',
        line=dict(width=2, color='red'),
        hoverinfo='skip'
    ))

    # Add a trend line using Plotly's trendline capabilities
    fig.add_trace(go.Scatter(
        x=df['Date'],
        y=df['Count'],  # Fit trend using Count vs Date
        mode='lines',
        line=dict(dash='dash', color='green'),
        name='Trend'
    ))

    # Find max and min points
    max_point = df.loc[df['Count'].idxmax()]
    min_point = df.loc[df['Count'].idxmin()]

    # Add annotations for max and min points
    fig.add_trace(go.Scatter(
        x=[max_point['Date']],
        y=[max_point['Count']],
        mode='markers+text',
        text=[f"Max: {max_point['Count']}"],
        textposition="top right",
        marker=dict(color='blue', size=10),
        name='Max Point'
    ))

    fig.add_trace(go.Scatter(
        x=[min_point['Date']],
        y=[min_point['Count']],
        mode='markers+text',
        text=[f"Min: {min_point['Count']}"],
        textposition="bottom right",
        marker=dict(color='orange', size=10),
        name='Min Point'
    ))

    # Customize layout and axis labels
    fig.update_layout(
        title=f'Time Series of Count over Days in {city_name}',
        xaxis_title='Date',
        yaxis_title='Count',
        legend_title='Legend',
        template='plotly_white',  # Clean template with gridlines
        xaxis=dict(
            tickangle=45,
            dtick=1,  # Set a daily tick interval
            showgrid=True
        ),
        yaxis=dict(
            showgrid=True
        ),
        hovermode='x unified'  # Better hover display
    )

    plot_json = fig.to_json()
    return  plot_json

In [21]:
@app.route('/kolkata_data', methods=['GET'])
def fetch_kolkata_data():
    try:
        # Connect to the database
        conn = connect_db()
        cursor = conn.cursor()
        
        # Execute the SQL query
        cursor.execute("""
            SELECT "ID", "Date", "Count", "Region ID", "Disease ID", "Disease", "Region"
            FROM "Kolkata";
        """)
        
        # Fetch the data and load it into a DataFrame
        kolkata_data = pd.DataFrame(cursor.fetchall(), columns=['ID', 'Date', 'Count', 'Region ID', 'Disease ID', 'Disease', 'Region'])
        fig_json = create_enhanced_time_series_plot(kolkata_data,"Kolkata")
        
        # Convert the DataFrame to JSON and return it
        return jsonify({'plot': fig_json})
    
    except Exception as e:
        return str(e)
    
    finally:
        cursor.close()
        conn.close()



In [22]:
@app.route('/delhi_data', methods=['GET'])
def fetch_delhi_data():
    try:
        # Connect to the database
        conn = connect_db()
        cursor = conn.cursor()
        
        # Execute the SQL query
        cursor.execute("""
            SELECT "ID", "Date", "Count", "Region ID", "Disease ID", "Disease", "Region"
            FROM "Delhi";
        """)
        
        # Fetch the data and load it into a DataFrame
        delhi_data = pd.DataFrame(cursor.fetchall(), columns=['ID', 'Date', 'Count', 'Region ID', 'Disease ID', 'Disease', 'Region'])
        fig_json = create_enhanced_time_series_plot(delhi_data,"Delhi")
        
        # Convert the DataFrame to JSON and return it
        return jsonify({'plot': fig_json})
    
    except Exception as e:
        return str(e)
    
    finally:
        cursor.close()
        conn.close()




In [23]:
@app.route('/pune_data', methods=['GET'])
def fetch_pune_data():
    try:
        # Connect to the database
        conn = connect_db()
        cursor = conn.cursor()
        
        # Execute the SQL query
        cursor.execute("""
            SELECT "ID", "Date", "Count", "Region ID", "Disease ID", "Disease", "Region"
            FROM "Pune";
        """)
        
        # Fetch the data and load it into a DataFrame
        pune_data = pd.DataFrame(cursor.fetchall(), columns=['ID', 'Date', 'Count', 'Region ID', 'Disease ID', 'Disease', 'Region'])
        fig_json = create_enhanced_time_series_plot(pune_data,"Pune")
        
        # Convert the DataFrame to JSON and return it
        return jsonify({'plot': fig_json})
    
    except Exception as e:
        return str(e)
    
    finally:
        cursor.close()
        conn.close()




In [16]:
if __name__ == '__main__':
    app.run(port=3009)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3009
Press CTRL+C to quit
127.0.0.1 - - [01/Oct/2024 17:34:09] "GET /kolkata_data HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/2024 17:34:10] "GET /kolkata_data HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/2024 17:34:12] "GET /kolkata_data HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/2024 17:34:13] "GET /kolkata_data HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/2024 17:34:15] "GET /kolkata_data HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/2024 17:34:16] "GET /kolkata_data HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/2024 17:34:18] "GET /delhi_data HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/2024 17:34:19] "GET /delhi_data HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/2024 17:34:21] "GET /delhi_data HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/2024 17:34:22] "GET /delhi_data HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/2024 17:34:23] "GET /delhi_data HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/2024 17:34:25] "GET /delhi_data HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/2024 17:34:26] "GET /pune_data HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/